### Imports

In [1]:
import numpy as np
import pandas as pd 
from collections import Counter
import os
import glob
import copy
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn import metrics
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from combat.pycombat import pycombat
from matplotlib.backends.backend_pdf import PdfPages
import math

### Read every cohort study file

In [2]:
datasets = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../cohort_studies_full_data/' + "/*."+'csv'))]
cohorts = [file.split(".")[0] for file in sorted(os.listdir('../cohort_studies_full_data/'))]

In [3]:
# make a dictionary that contains all cohorts as a dataframe
cohort_studies = dict()
# dfsss = dict()

for cohort, dataset in zip(cohorts, datasets):
    cohort_n = cohort.split("_MERGE")[0]
    cohort_studies[cohort_n] = dataset.loc[dataset['Months']==0].copy() # reduce to BL visit
#     dfsss[cohort_n] = dataset

In [4]:
datasets_sub = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../preprocessed_datasets/' + "/*."+'csv'))]
cohorts_sub = [file.split(".")[0] for file in sorted(os.listdir('../preprocessed_datasets/'))]

In [5]:
# make a dictionary that contains all cohorts as a dataframe
cohort_studies_sub = dict()

for cohort, dataset in zip(cohorts_sub, datasets_sub):
    cohort_studies_sub[cohort] = dataset.loc[dataset['Months']==0].copy() # reduce to BL visit

In [6]:
# add the preprocessed columns from sub table of each dataset to the main table of dataset
for i in cohort_studies:
#     cols = cohort_studies_sub[i].columns.difference(cohort_studies[i].columns)
    cols = ['Age', 'Sex', 'Education', 'APOE4', 'CDR', 'Race']
    
    for col in cols:
        
        if col in cohort_studies_sub[i].columns:
            cohort_studies[i][col] = cohort_studies_sub[i][col]

### Read harmonized mapping tables

In [7]:
modality = [pd.read_csv(file, sep=',') for file in sorted(glob.glob('../feature_tables' + "/*."+'csv'))]
name = [file.split(".")[0] for file in sorted(os.listdir('../feature_tables'))]

In [8]:
# make a dictionary that contains all modalities as a dataframe
mappings = dict()

for moda, na in zip(modality, name):
    mappings[na.split(' - ')[1]] = moda

In [9]:
harmonized_features = pd.concat(mappings, ignore_index=True) # combine all tables

In [10]:
# exclude categorical and taboo features
harmonized_features = harmonized_features.loc[(harmonized_features['Rank']!=1) & (harmonized_features['Rank']!=2)]

### Read the feature availability files for all cohorts

In [11]:
ava_mapp = [pd.read_csv(file, sep='\t') for file in sorted(glob.glob('../feature_availability_in_cohorts' + "/*."+'tsv'))]
tablesss = [file.split(".")[0] for file in sorted(os.listdir('../feature_availability_in_cohorts'))]

In [12]:
# make a dictionary that contains all modalities as a dataframe
available_features = dict()

for modal, df in zip(tablesss, ava_mapp):
    available_features[modal] = df

In [13]:
existing_features = pd.concat(available_features, ignore_index=True) # combine all tables

In [14]:
existing_features.replace({0: np.nan}, inplace=True) # 0 indicates that the feature was not measured 

### Read cutoffs obtained using all methods

In [15]:
table_method = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/cutoffs/' + "/*."+'csv'))]
method_name = [file.split(".")[0] for file in sorted(os.listdir('../results/cutoffs/'))]

In [16]:
# make a dictionary that contains each cutoff table as a dataframe
cutoffs_ = dict()

for tm, mn in zip(table_method, method_name):
    cutoffs_[mn] = tm

In [17]:
[cutoffs_['km_cutoffs'].rename(columns={col: col.split('_')[0]}, inplace=True) for col in cutoffs_['km_cutoffs'].columns]

[None, None, None]

### Selecetion of cohort studies for A/T/N assignment

### Select the patient that have CSF biomarker, disregard the diagnostic status

In [18]:
atn = pd.DataFrame(index=available_features['csf'].iloc[:3].replace({0: np.nan}).dropna(axis=1).columns[1:].to_list(), columns=mappings['csf'].Feature.loc[0:2].to_list()+(["Total"]))
# atn = pd.DataFrame(index=cohort_studies, columns=['A', 'T', 'N'])

In [19]:
for cohort in atn.index:
    for feat in mappings['csf'][cohort].loc[0:2].dropna().to_list():
        if feat in cohort_studies[cohort].columns:
            atn.loc[cohort, mappings['csf'].loc[mappings['csf'][cohort]==feat, 'Feature']] = len(cohort_studies[cohort][feat].dropna())
            atn.loc[cohort, 'Total'] = len(cohort_studies[cohort][mappings['csf'][cohort].loc[0:2].dropna().to_list()].dropna())

In [20]:
# atn

In [21]:
diag = pd.DataFrame(index=available_features['csf'].iloc[:3].replace({0: np.nan}).dropna(axis=1).columns[1:].to_list(), columns=cohort_studies['ADNI']['Diagnosis'].dropna().unique())

In [22]:
for cohort in diag.index:
    for dia in diag.columns:
        diag.loc[cohort, dia] = len(cohort_studies[cohort].loc[cohort_studies[cohort]['Diagnosis']==dia][mappings['csf'][cohort].loc[0:2].dropna().to_list()].dropna())

### Remove the empty columns from all cohorts that we are intrested in
### Remove the participant without all 3 CSF biomarkers

In [23]:
selected_cohorts = dict()

for coh in diag.index:
    selected_cohorts[coh] = cohort_studies[coh].dropna(axis=1, how='all')

In [24]:
total_feats = dict()

# existing_features.set_index('Feature', inplace=True)

for feat in existing_features.Feature:
    total_feats[feat] = existing_features.loc[existing_features.Feature==feat][selected_cohorts].dropna(axis=1).columns

In [25]:
for cohort in atn.index:
    feat = mappings['csf'][cohort].loc[0:2].dropna().to_list()
    cohort_studies[cohort] = cohort_studies[cohort].dropna(subset=feat)

As Some features have suffix due to merging tables for certain cohorts, first investigate if all the harmonized features are in cohorts. Rename the ones that have suffix so it can be compatible to work with our harmonized names.

In [26]:
cohort_studies['ADNI'] = cohort_studies['ADNI'].rename(columns={'PTEDUCAT_x': 'PTEDUCAT', 'TRABSCOR_bl': 'TRABSCOR', 'LDELTOTAL_BL': 'LDELTOTAL'})

### CSF biomarkers, two classes, normal vs abnormal

In [27]:
# modality_ = ['clinical_i', 'clinical_ii','hippocampus', 'csf']
# modality_ = ['basal_ganglia', 'brain_poles_volume', 'cerebellum', 'clinical_i', 
#             'clinical_ii', 'csf', 'diencephalus', 'general_brain', 
#             'mri_others', 'pet', 'plasma', 'ventricles', 'hippocampus']
modality_ = ['basal_ganglia', 'brain_poles_volume', 'cerebellum', 'csf', 'diencephalus', 'general_brain', 
            'mri_others', 'pet', 'plasma', 'ventricles', 'hippocampus']

selected_feat = dict()

for i in modality_:
    selected_feat[i] = mappings[i]

In [28]:
# make a dataframe containing all the mapped features
features_all = pd.concat(selected_feat, ignore_index=True)

In [29]:
features_all = features_all[atn.index.union(['Feature', 'Rank'])] # subset the cohorts of interest

In [30]:
# remove the features that are not available in all studies
features_all = features_all.loc[features_all['Feature'].isin(existing_features[atn.index.union(['Feature'])].dropna(how='all')['Feature'].to_list())]

### Convert MRI measurements to mm3, same as ADNI and other cohorts

In [31]:
mri_nacc = ['basal_ganglia', 'brain_poles_volume', 'cerebellum', 'diencephalus', 'general_brain', 
            'mri_others', 'pet', 'plasma', 'ventricles', 'hippocampus']

for i in mri_nacc:
    
    for variable in mappings[i][['Feature', 'NACC']].dropna()['Feature'].to_list():
        
        if "Volume" in variable:
            nacc_var = mappings[i].loc[mappings[i]['Feature']==variable, 'NACC']
            cohort_studies['NACC'][nacc_var] = cohort_studies['NACC'][nacc_var] * 1000

In [32]:
nacc_mri_to_convert = list()

for i in mri_nacc:
    
    for feat_ in mappings[i]['NACC'].dropna().to_list():
        nacc_mri_to_convert.append(feat_)

In [33]:
# cohort_studies['NACC'][nacc_mri_to_convert].dropna(how='all')

* Rank 1 --> Categorical features
* Rank 2 --> Taboo features: some categorical and some numerical
* Rank nan --> Numerical features 

replace nan with 0

In [34]:
features_all['Rank'].replace({np.nan: 0}, inplace=True)

In [35]:
features_all.replace({"No total score.": np.nan}, inplace=True)

In [36]:
# fix PharmaCog column names
for i in cohort_studies['PharmaCog'].columns:
    if "\xa0" in i:
        new = str(i).replace(u'\xa0', u'')
        cohort_studies['PharmaCog'].rename(columns={i: new}, inplace=True)

In [37]:
Counter(cohort_studies['PharmaCog']['Diagnosis'])

Counter({'MCI': 145})

## Prepare datasets

In [38]:
def select_atn_participants(dfss, cohorts, thresholds, features):
    """cohorts: list of cohort names 
       dfss: dictionary of cohorts where each key is the name of a cohort
       thresholds: cutoff values obtained using a methodology
       
       Select the features and participants and categorize the participant into ATN profiles using thresholds
       obtained from each methodology.
       
       return a df which contain the combination of paticipant from the selected cohorts while harmonizing
       the features names.
    """
    
    # make a list of additional features to be investigated 
#     additional_feat = ['Age', 'Sex', 'Education', 'APOE4', 'CDR', 'Race']
    additional_feat = ['APOE4']

    # make an empty dictionary to add the datasets to
    df_= pd.DataFrame(columns=set(features.loc[features['Rank']==0]
                                  [cohorts + ['Feature']].dropna()['Feature']).difference(['Feature'] + list(thresholds.columns)).union(additional_feat))
   
    for i in cohorts:
#         print(i)

        dfs = dict()
        dfs[i] = dfss[i].copy() # make a copy of the dataset of interest
        # select the subset of datasets with features to be investigated
        dfs[i] = dfs[i][features.loc[features['Rank']==0][cohorts + ['Feature']].dropna()[i].to_list() + additional_feat].dropna(axis=1, how='all')
        # rename all the columns so we can concat the datasets later
        [dfs[i].rename(columns={col: coln}, inplace=True) for col, coln in 
         zip(features.loc[features['Rank']==0][cohorts + ['Feature']].dropna()[i].to_list(), 
             features.loc[features['Rank']==0][cohorts + ['Feature']].dropna()['Feature'].to_list())]
        dfs[i]['Cohort'] = i # add a cohort name column
        # change the datatype to str as these are categorical features. astype doesn't work as it will include nan values
        dfs[i] = dfs[i].replace({'APOE4': {0.0: '0', 2.0: '2', 1.0: '1'}}) 

        if i!='NACC':
            
            for biomarker in thresholds.columns:
                # select the cutoff value for each biomarker for each cohort
                threshold = thresholds.loc[i][biomarker]

                # dichotomize the participants
                if biomarker == 'pTau in CSF': 
                    dfs[i].loc[dfs[i][biomarker]>threshold, "T"] = 'T+'
                    dfs[i].loc[dfs[i][biomarker]<threshold, "T"] = 'T-'

                elif biomarker == 'tTau in CSF': 
                    dfs[i].loc[dfs[i][biomarker]>threshold, "N"] = 'N+'
                    dfs[i].loc[dfs[i][biomarker]<threshold, "N"] = 'N-'

                else: 
                    dfs[i].loc[dfs[i][biomarker]<threshold, "A"] = 'A+'
                    dfs[i].loc[dfs[i][biomarker]>threshold, "A"] = 'A-'

            # join the 3 columns to make the final ATN categorie                                 
            dfs[i]['ATN'] = dfs[i]['A'] + dfs[i]['T'] + dfs[i]['N']
            # remove the columns that we are not interested in
            dfs[i] = dfs[i][dfs[i].columns.difference(['A', 'T', 'N', 'Mini-Mental State Examination (MMSE)'] + list(thresholds.columns))]
            
        else:
            
            elisa_index = cohort_studies[i].loc[(cohort_studies[i]['CSFTTMD']==1)].index
            xmap_index = cohort_studies[i].loc[(cohort_studies[i]['CSFTTMD']==2)].index
            
            for biomarker in thresholds.columns:
                # select the cutoff value for each biomarker for each cohort
                elisa = thresholds.loc[i + "_ELISA"][biomarker] # ELISA
                xmap = thresholds.loc[i + "_XMAP"][biomarker] #XMAP

                # dichotomize the participants
                if biomarker == 'pTau in CSF': 
                    
                    dfs[i].loc[(dfs[i].index.isin(elisa_index)) & (dfs[i][biomarker]>elisa), "T"] = 'T+'
                    dfs[i].loc[(dfs[i].index.isin(elisa_index)) & (dfs[i][biomarker]<elisa), "T"] = 'T-'
                    dfs[i].loc[(dfs[i].index.isin(xmap_index)) & (dfs[i][biomarker]>xmap), "T"] = 'T+'
                    dfs[i].loc[(dfs[i].index.isin(xmap_index)) & (dfs[i][biomarker]<xmap), "T"] = 'T-'

                elif biomarker == 'tTau in CSF': 
                    dfs[i].loc[(dfs[i].index.isin(elisa_index)) & (dfs[i][biomarker]>elisa), "N"] = 'N+'
                    dfs[i].loc[(dfs[i].index.isin(elisa_index)) & (dfs[i][biomarker]<elisa), "N"] = 'N-'
                    dfs[i].loc[(dfs[i].index.isin(xmap_index)) & (dfs[i][biomarker]>xmap), "N"] = 'N+'
                    dfs[i].loc[(dfs[i].index.isin(xmap_index)) & (dfs[i][biomarker]<xmap), "N"] = 'N-'

                else: 
                    dfs[i].loc[(dfs[i].index.isin(elisa_index)) & (dfs[i][biomarker]<elisa), "A"] = 'A+'
                    dfs[i].loc[(dfs[i].index.isin(elisa_index)) & (dfs[i][biomarker]>elisa), "A"] = 'A-'
                    dfs[i].loc[(dfs[i].index.isin(xmap_index)) & (dfs[i][biomarker]<xmap), "A"] = 'A+'
                    dfs[i].loc[(dfs[i].index.isin(xmap_index)) & (dfs[i][biomarker]>xmap), "A"] = 'A-'

            # join the 3 columns to make the final ATN categorie                                 
            dfs[i]['ATN'] = dfs[i]['A'] + dfs[i]['T'] + dfs[i]['N']
            # remove the columns that we are not interested in
            dfs[i] = dfs[i][dfs[i].columns.difference(['A', 'T', 'N', 'Mini-Mental State Examination (MMSE)', 'Trail Making Test (TMT) A', 'Verbal fluency tests (Semantic) Animal'] + list(thresholds.columns))]
            
#         print(dfs[i])

        
        df_ = pd.concat([df_, dfs[i]])
#         print(df_)
#         df_ = df_.dropna(axis=1, thresh=len(df_.index)/2)
#         print(df_)
    
    df_ = df_.dropna(axis=1, how='all')
    df_ = df_.dropna(axis=1, thresh=1200)

    df_ = df_.dropna()

    return df_

In [39]:
clustering_df_km = select_atn_participants(cohort_studies, ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog'], cutoffs_['km_cutoffs'], features_all)
clustering_df_gmm = select_atn_participants(cohort_studies, ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog'], cutoffs_['gmm_cutoffs'], features_all)

#### exclude CSF Volume as it seems wrong in the NACC dataset and that could potentially add bias to our analysis

In [40]:
clustering_df_km = clustering_df_km[clustering_df_km.columns.difference(['Cerebrospinal Fluid Volume'])]
clustering_df_gmm = clustering_df_gmm[clustering_df_gmm.columns.difference(['Cerebrospinal Fluid Volume'])]

#### Batch effect correstion

In [41]:
km_corrected = pycombat(clustering_df_km.drop(columns=['ATN', 'Cohort', 'APOE4']).transpose(), 
         batch=clustering_df_km['Cohort']).transpose()

gmm_corrected = pycombat(clustering_df_gmm.drop(columns=['ATN', 'Cohort', 'APOE4']).transpose(), 
         batch=clustering_df_gmm['Cohort']).transpose()

Found 7 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.
Adjusting the Data
Found 7 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.
Adjusting the Data


/usr/lib64/python3.9/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [42]:
km_corrected[['ATN', 'Cohort', 'APOE4']] = clustering_df_km[['ATN', 'Cohort', 'APOE4']]
gmm_corrected[['ATN', 'Cohort', 'APOE4']] = clustering_df_gmm[['ATN', 'Cohort', 'APOE4']]

In [43]:
gmm_corrected.to_csv("../results/batch_effect_corrected_dfs/gmm.csv")
km_corrected.to_csv("../results/batch_effect_corrected_dfs/km.csv")

In [44]:
# with PdfPages('../results/boxplot_batch_correction/batch_effect.pdf') as pdf:

#     for i in clustering_df_km.columns.difference(['ATN', 'APOE4', 'Cohort']):
#         fig, axes = plt.subplots(1,2, figsize=(20, 8))

#         sns.boxplot(data=clustering_df_km, x='Cohort', y=i, ax=axes[0])
#         sns.boxplot(data=km_corrected, x='Cohort', y=i, ax=axes[1])
#         axes[0].set_title("Raw measurements")
#         axes[1].set_title("Corrected measurements")

#         fig.get_figure()
#         pdf.savefig(fig)

In [45]:
# with PdfPages('../results/boxplot_batch_correction/batch_effect_gmm.pdf') as pdf:

#     for i in clustering_df_km.columns.difference(['ATN', 'APOE4', 'Cohort']):
#         fig, axes = plt.subplots(1,2, figsize=(20, 8))

#         sns.boxplot(data=clustering_df_gmm, x='Cohort', y=i, ax=axes[0])
#         sns.boxplot(data=gmm_corrected, x='Cohort', y=i, ax=axes[1])
#         axes[0].set_title("Raw measurements")
#         axes[1].set_title("Corrected measurements")

#         fig.get_figure()
#         pdf.savefig(fig)

In [77]:
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

In [78]:
def atn_based_df(df):
    """"""
    
    labels_atn = dict()
    score_atn = dict()
    
    # change the cohort names to int and write in a column
    for i, j in zip(df.Cohort.unique(), range(len(df.Cohort.unique()))):
        df.loc[df['Cohort']==i, 'Cohort_number'] = j
    
    # select the profiles that have over 20 participants
    # make a dictionary where the selcted ATN profiles are key and 
    # the subset of dataframe categorized in that ATN profile is value
    dfs_sub = {atn: df.loc[df.ATN==atn].copy() for atn in list((a) for a,b in dict(Counter(df.ATN)).items() if b >20)}
    
    for i in dfs_sub:
#         print(i, len(dfs_sub[i].Cohort.unique()))
        hierarchical_cluster = AgglomerativeClustering(n_clusters=len(dfs_sub[i].Cohort.unique()), affinity='euclidean', linkage='ward')
        labels = hierarchical_cluster.fit_predict(dfs_sub[i][dfs_sub[i].columns.difference(['ATN', 'Cohort', 'Cohort_number'])])
        labels_atn[i] = labels
        score_atn[i] = purity_score(dfs_sub[i]['Cohort_number'], labels)
     
    return labels_atn, score_atn, dfs_sub

In [79]:
labels_km, score_km, atn_km_df = atn_based_df(clustering_df_km)
labels_gmm, score_gmm, atn_gmm_df = atn_based_df(clustering_df_gmm)

In [80]:
#test
labels_km_t, score_km_t, atn_km_df_t = atn_based_df(km_corrected)
labels_gmm_t, score_gmm_t, atn_gmm_df_t = atn_based_df(gmm_corrected)

In [81]:
# print("Cluster purity (K-means):")
# score_km

In [82]:
# print("Cluster purity (GMM):")
# score_gmm

In [83]:
for i in labels_km:
    atn_km_df[i].loc[:, 'predicted_cohort'] = list(labels_km[i])

for i_ in labels_gmm:
    atn_gmm_df[i_].loc[:, 'predicted_cohort'] = list(labels_gmm[i_])

In [84]:
#test
for i in labels_km_t:
    atn_km_df_t[i].loc[:, 'predicted_cohort'] = list(labels_km_t[i])

for i_ in labels_gmm_t:
    atn_gmm_df_t[i_].loc[:, 'predicted_cohort'] = list(labels_gmm_t[i_])

In [85]:
def calculate_cramer(dfs):
    """ """
    results = dict()
    
    for i in dfs:
        df_ = dfs[i].set_index('Cohort')
        mat = pd.crosstab(df_.index, df_['predicted_cohort'])
#         print(i, mat)
#         print(i, round(stats.contingency.association(mat, method='cramer'), 2))
        results[i] = round(stats.contingency.association(mat, method='cramer'), 2)
        
    return results

In [86]:
# pd.DataFrame.from_dict(calculate_cramer(atn_km_df), orient='index').transpose()[['A-T-N-', 'A-T+N+', 'A-T-N+', 'A+T+N-', 'A+T-N-', 'A+T-N+', 'A+T+N+']]

In [87]:
# pd.DataFrame.from_dict(calculate_cramer(atn_gmm_df), orient='index').transpose()[['A-T-N-', 'A-T+N+', 'A-T-N+', 'A+T+N-', 'A+T-N-', 'A+T-N+', 'A+T+N+']]

### checking the batch-corrected ones

In [96]:
cramer_km = pd.DataFrame.from_dict(calculate_cramer(atn_km_df_t), orient='index').transpose()[['A-T-N-', 'A-T+N+', 'A-T-N+', 'A+T+N-', 'A+T-N-', 'A+T-N+', 'A+T+N+']]

In [97]:
cramer_gmm = pd.DataFrame.from_dict(calculate_cramer(atn_gmm_df_t), orient='index').transpose()[['A-T-N-', 'A-T+N+', 'A-T-N+', 'A+T+N-', 'A+T-N-', 'A+T-N+', 'A+T+N+']]

In [118]:
for i,j in zip([cramer_km, cramer_gmm], ['K-Means', 'GMM']):
    i.rename(index={0: j}, inplace=True)
    i.to_csv(f"../results/clustering/cramers_v/cramers_v_{j}.csv")

In [119]:
def calculate_chi2(dfs):
    """ """
    results_chi2 = dict()
    results_p = dict()
    results_df = dict()
    results_ex = dict()
    
    for i in dfs:
        df_ = dfs[i].set_index('Cohort')
        mat = pd.crosstab(df_.index, df_['predicted_cohort'])
        results_chi2[i], results_p[i], results_df[i], results_ex[i] = stats.chi2_contingency(mat, correction=True)
        
    return results_chi2, results_p, results_df, results_ex

In [120]:
# results_chi2_km, results_p_km, results_df_km, results_ex_km = calculate_chi2(atn_km_df)
# results_chi2_gmm, results_p_gmm, results_df_gmm, results_ex_gmm = calculate_chi2(atn_gmm_df)

# corrected dataframes
results_chi2_km_t, results_p_km_t, results_df_km_t, results_ex_km_t = calculate_chi2(atn_km_df_t)
results_chi2_gmm_t, results_p_gmm_t, results_df_gmm_t, results_ex_gmm_t = calculate_chi2(atn_gmm_df_t)

In [150]:
results_chi2_gmm_t

{'A-T-N-': 47.2738997828431,
 'A+T-N-': 39.86569318064662,
 'A+T+N+': 50.74756679162755,
 'A-T+N+': 12.767045454545453,
 'A+T+N-': 24.40691661279897,
 'A+T-N+': 33.557491522843705,
 'A-T-N+': 25.86746031746032}

In [121]:
pvalue_km = pd.DataFrame.from_dict(results_p_km_t, orient='index').transpose()[['A-T-N-', 'A-T+N+', 'A-T-N+', 'A+T+N-', 'A+T-N-', 'A+T-N+', 'A+T+N+']].round(2)

In [122]:
pvalue_gmm = pd.DataFrame.from_dict(results_p_gmm_t, orient='index').transpose()[['A-T-N-', 'A-T+N+', 'A-T-N+', 'A+T+N-', 'A+T-N-', 'A+T-N+', 'A+T+N+']].round(2)

### Combine Cramer's V and P-values

In [123]:
pvalue_gmm

,A-T-N-,A-T+N+,A-T-N+,A+T+N-,A+T-N-,A+T-N+,A+T+N+
0,0.1,0.69,0.41,0.08,0.3,0.12,0.05


In [124]:
for i,j in zip([pvalue_km, pvalue_gmm], ['K-Means', 'GMM']):
    i.rename(index={0: j}, inplace=True)
    i.to_csv(f"../results/clustering/p_value/pvalue_{j}.csv")

### Note: One could use the crosstab instead of the function below

In [125]:
def count_clusters(df):
    
    """df: dataframe with ATN categorization using certain method and containing cluster labels
       return: the number of participant within each cohort and within each ATN profile assigned to each labels"""
    
    # make an empty dictionary of dataframes to store the results
    clustering_result_ = {i: pd.DataFrame(index=df[i].Cohort.unique(), 
                                            columns=sorted(df[i].predicted_cohort.unique())) for i in df}
    
    # check the number of participants clustered to each coohort within each biomarker profile
    for profi in df:
    
        for name in df[profi]['Cohort'].unique():
            clustering_result_[profi].loc[name] = Counter(df[profi].loc[df[profi]['Cohort']==name, 'predicted_cohort'])
            
            
    # replace all nan enteries with 0
    [clustering_result_[i].replace({np.nan: 0}, inplace=True) for i in clustering_result_]
    
    # change enteries to integer
    for i in clustering_result_:
        clustering_result_[i] = clustering_result_[i].astype(int)

    return clustering_result_

In [126]:
clustering_result_km = count_clusters(atn_km_df_t)
clustering_result_gmm = count_clusters(atn_gmm_df_t)

In [183]:
print("K-Means")
for i in ['A-T-N-', 'A-T+N+', 'A-T-N+', 'A+T+N-', 'A+T-N-', 'A+T-N+', 'A+T+N+']:
    print(i, len(clustering_result_km[i].index), list(clustering_result_km[i].index))
    
print("GMM")
for i in ['A-T-N-', 'A-T+N+', 'A-T-N+', 'A+T+N-', 'A+T-N-', 'A+T-N+', 'A+T+N+']:
    print(i, len(clustering_result_gmm[i].index), list(clustering_result_gmm[i].index))

K-Means
A-T-N- 7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
A-T+N+ 6 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'DOD-ADNI', 'PharmaCog']
A-T-N+ 6 ['ADNI', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
A+T+N- 7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
A+T-N- 7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
A+T-N+ 6 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'PharmaCog']
A+T+N+ 7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
GMM
A-T-N- 7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
A-T+N+ 5 ['ADNI', 'ARWIBO', 'NACC', 'DOD-ADNI', 'PharmaCog']
A-T-N+ 6 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'PharmaCog']
A+T+N- 5 ['ADNI', 'EDSD', 'NACC', 'JADNI', 'PharmaCog']
A+T-N- 7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', 'PharmaCog']
A+T-N+ 6 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'PharmaCog']
A+T+N+ 7 ['ADNI', 'EDSD', 'ARWIBO', 'NACC', 'JADNI', 'DOD-ADNI', '

In [128]:
clustering_result_km.keys()

dict_keys(['A-T-N-', 'A+T-N-', 'A+T+N+', 'A-T+N+', 'A+T+N-', 'A-T-N+', 'A+T-N+'])

In [129]:
clustering_result_gmm.keys()

dict_keys(['A-T-N-', 'A+T-N-', 'A+T+N+', 'A-T+N+', 'A+T+N-', 'A+T-N+', 'A-T-N+'])

In [149]:
clustering_result_gmm['A+T-N+']

,0,1,2,3,4,5
ADNI,3,2,0,1,0,2
EDSD,2,4,0,1,1,0
ARWIBO,2,4,2,3,0,3
NACC,9,4,6,2,5,1
JADNI,0,3,1,0,0,1
PharmaCog,0,1,0,0,1,3


In [131]:
for i in clustering_result_gmm:
    clustering_result_gmm[i].to_csv(f"../results/clustering/gmm/{i}_gmm.csv")
    clustering_result_km[i].to_csv(f"../results/clustering/k_means/{i}_km.csv")

### Cramers' V interpretation based on degrees of freedom
#### Degrees of freedom is calculated --> min(row-1, column-1)

In [160]:
cramers_df = pd.DataFrame(index=list(range(1,8)), columns=['s', 'm', 'l'])

In [162]:
cramers_df.iloc[0] = [0.1, 0.3, 0.5]

In [171]:
cramers_df.iloc[0]['m']

0.3

In [176]:
for i in cramers_df.iloc[1:].index:
    
    for col in cramers_df.columns:
        
        cramers_df.loc[i, col] = round(cramers_df.iloc[0][col] / math.sqrt(i), 3)

In [177]:
cramers_df

,s,m,l
1,0.1,0.3,0.5
2,0.071,0.212,0.354
3,0.058,0.173,0.289
4,0.05,0.15,0.25
5,0.045,0.134,0.224
6,0.041,0.122,0.204
7,0.038,0.113,0.189
